<a href="https://colab.research.google.com/github/hyun-hyang/ResumEmphasizer/blob/main/resume_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [162]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [163]:
spacy.__version__

'3.4.1'

In [164]:
!nvidia-smi

Mon Aug  8 20:57:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [165]:
!git clone 'https://github.com/hyun-hyang/ResumEmphasizer.git'

fatal: destination path 'ResumEmphasizer' already exists and is not an empty directory.


In [166]:
%cd /content/ResumEmphasizer
!git pull

/content/ResumEmphasizer
Already up to date.


In [167]:
cv_data = json.load(open('/content/ResumEmphasizer/Data_annoctated/merged res.json','r'))

In [168]:
len(cv_data)

99

In [169]:
!python -m spacy init fill-config base_config.cfg config.cfg

Usage: python -m spacy init fill-config 
           [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config.cfg' does not exist.


In [170]:
!python -m spacy init fill-config /content/ResumEmphasizer/Data/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [171]:
cv_data

[{'id': 297,
  'label': [[0, 13, 'ability'],
   [623, 655, 'work experience'],
   [671, 724, 'work experience'],
   [1202, 1249, 'work experience'],
   [1274, 1349, 'work experience'],
   [1399, 1437, 'work experience'],
   [1600, 1655, 'award'],
   [1686, 1712, 'major'],
   [1713, 1716, 'degree'],
   [1725, 1757, 'college'],
   [1900, 1921, 'education'],
   [2020, 2026, 'skill'],
   [2027, 2034, 'skill'],
   [2035, 2041, 'skill'],
   [2058, 2074, 'skill'],
   [2075, 2095, 'skill'],
   [2097, 2100, 'skill'],
   [2101, 2106, 'skill'],
   [2108, 2112, 'skill'],
   [2113, 2123, 'skill'],
   [2125, 2129, 'skill'],
   [2130, 2133, 'skill'],
   [2135, 2138, 'skill'],
   [2165, 2182, 'ability'],
   [2183, 2198, 'ability'],
   [2199, 2215, 'ability'],
   [2216, 2229, 'ability'],
   [2336, 2363, 'skill'],
   [2402, 2420, 'skill'],
   [2456, 2477, 'skill']],
  'text': 'Web Developer Resume Template (Full Text Version)\nOlivier Baudet\nAddress: 44 place de Miremont, 77170, Paris, France\nEmail ad

In [172]:
#from re import T
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for item in data:
    text = item['text']
    doc = nlp.make_doc(text)
    annot = item['label']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end , label=label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)

      else:
        ents.append(span)
      
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

    return db
      

In [173]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data , test_size=0.3)

In [174]:
len(train), len(test)

(69, 30)

In [175]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,train)
db.to_disk('test_data.spacy')

file.close()

In [176]:
len(db.tokens)

1

In [177]:
!python -m spacy train /content/ResumEmphasizer/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-08-08 20:57:42,574] [INFO] Set up nlp object from config
[2022-08-08 20:57:42,585] [INFO] Pipeline: ['transformer', 'ner']
[2022-08-08 20:57:42,589] [INFO] Created vocabulary
[2022-08-08 20:57:42,590] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a 

Model Test

In [186]:
nlp = spacy.load('/content/ResumEmphasizer/output/model-best')


In [185]:
doc = nlp('my name is Leo. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text, "    ->>>>  ", ent.label_)

In [180]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [181]:
import sys, fitz

In [193]:
# fname = '/content/ResumEmphasizer/test_cv/3.pdf'
# doc = fitz.open(fname)

# text = " "
# for page in doc:
#   text = text + str(page.get_text())

# text = text.strip()
# text = ' '.join(text.split())

# doc = nlp(text)
# for ent in doc.ents:
#   print(ent.text, "    ->>>>  ", ent.label_)

•     ->>>>   ability
Worked as a Software Developer in IBM, Hyderabad for 1 year from May 2016 to May 2017 • Developed Application Information Document (AID), which will provide members of the team with an overview of the Dev Automation and also tested 55 modules. • Programing skills: C-Programming, Core Java, Advanced Java, HTML, AJWT Education B.Tech in Computer Science Engineering in Computer Science Engineering Perusing MBA form Pondicherry University Puducherry, Puducherry Skills ANDROID (Less than 1 year), FANUC (Less than 1 year), HELP DESK (Less than 1 year), Java (Less than 1 year), MS office (Less than 1 year) Additional Information • Computer skills: MS office, Operating System RELEVANT SKILLS Communication • Worked as Class Representative and organizing different college events. • Coordinating and Organizing Cultural activities & Anchoring the events in college Presentation Skills • Developed and delivered a Power Point Presentation on Graphics in c and digital clock in ja

In [196]:
for x in range(1,11):
  fname='/content/ResumEmphasizer/test_cv/'+str(x)+'.pdf'
  doc=fitz.open(fname)
  print('resume num '+str(x)+' test')
  text=" "
  for page in doc:
    text=text+str(page.get_text())
  text=text.strip()
  text=' '.join(text.split())
  
  doc=nlp(text)
  for ent in doc.ents:
    print(ent.text, " ->>>>>> ", ent.label_)
  print('\n')

resume num 1 test


resume num 2 test
•  ->>>>>>  ability
Java  ->>>>>>  skill
•  ->>>>>>  ability
JSP  ->>>>>>  skill
JDBC  ->>>>>>  skill
•  ->>>>>>  ability
•  ->>>>>>  ability
JDBC  ->>>>>>  skill
•  ->>>>>>  ability
web  ->>>>>>  skill
HTML  ->>>>>>  skill
Javascript  ->>>>>>  skill
•  ->>>>>>  ability
•  ->>>>>>  ability
C and  ->>>>>>  skill
C++ programming  ->>>>>>  skill
•  ->>>>>>  ability
•  ->>>>>>  ability
•  ->>>>>>  ability
•  ->>>>>>  ability
•  ->>>>>>  ability
•  ->>>>>>  ability
•  ->>>>>>  ability
Core Java  ->>>>>>  skill
JSP  ->>>>>>  skill
Spring  ->>>>>>  skill
JDBC  ->>>>>>  skill
Webservices  ->>>>>>  skill
•  ->>>>>>  ability
Java  ->>>>>>  skill
JSP  ->>>>>>  skill
Struts  ->>>>>>  skill
Spring  ->>>>>>  skill
Bachelor Of Engineering Mumbai  ->>>>>>  major
C  ->>>>>>  skill
Coding  ->>>>>>  skill
CSS  ->>>>>>  skill
database  ->>>>>>  skill
•  ->>>>>>  ability
Java  ->>>>>>  skill
SQL  ->>>>>>  skill
C++  ->>>>>>  skill
•  ->>>>>>  ability
Java  ->>>>>>  ski